# 14차시 보강: 전이학습 전략 비교 실습

이 실습에서는 **3가지 전이학습 전략**을 비교합니다:
1. **Freeze (전체 동결)**: 백본 전체를 고정하고 분류기만 학습
2. **Partial (부분 해제)**: 마지막 블록(layer4)과 분류기만 학습
3. **Full (전체 해제)**: 모든 층을 학습

**목표**: 소규모 데이터셋(3,000개)에서 각 전략의 성능을 비교하고 최적 전략을 찾기

In [ ]:
!sudo apt-get install -y fonts-nanum* | tail -n 1
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

In [ ]:
# 필요 라이브러리 설치

!pip install torchviz | tail -n 1
!pip install torchinfo | tail -n 1

런타임 다시 시작하세요.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib

# 나눔고딕 폰트 경로 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path)

# matplotlib 기본 폰트로 지정
matplotlib.rc('font', family='NanumGothic')

# 마이너스 부호 깨짐 방지
matplotlib.rcParams['axes.unicode_minus'] = False

print("한글 폰트 설정 완료:", matplotlib.rcParams['font.family'])

1. 라이브러리 임포트 및 환경설정

In [ ]:
# 필수 라이브러리 임포트
import torch  # PyTorch 메인 라이브러리
import torch.nn as nn  # 신경망 모듈
import torch.optim as optim  # 최적화 알고리즘
from torchvision import datasets, transforms, models  # 비전 관련 도구
from torch.utils.data import DataLoader, Subset  # 데이터 로더
import numpy as np  # 수치 연산
import random  # 랜덤 함수

# GPU 사용 가능 여부 확인 및 디바이스 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'사용 디바이스: {device}')

# 재현성을 위한 시드 설정 (매번 동일한 결과를 얻기 위함)
torch.manual_seed(0)  # PyTorch 시드
np.random.seed(0)  # NumPy 시드
random.seed(0)  # Python random 시드

# CUDA 사용 시 추가 재현성 설정
if torch.cuda.is_available():
    torch.cuda.manual_seed(0)  # CUDA 시드
    torch.backends.cudnn.deterministic = True  # 결정적 알고리즘 사용
    torch.backends.cudnn.benchmark = False  # 벤치마크 비활성화

print('환경 설정 완료!')
print(f'PyTorch 버전: {torch.__version__}')

데이터 전처리

In [ ]:
transform_train = transforms.Compose([
                  transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
                  # 이미지의 60-100% 영역을 무작위로 잘라 224*224 이미지로 조정
                  transforms.RandomHorizontalFlip(),
                  # 50%확률로 좌우 반전

                  transforms.ToTensor(),
                  transforms.Normalize(mean = (0.485, 0.456, 0.406),
                                      std = (0.229, 0.224, 0.225))
                  # RGB 각 채널의 평균과 표준편차
              ]
              )


transform_test = transforms.Compose([
                  transforms.Resize(256), # 이미지를 256*256 사이즈로 변형
                  transforms.CenterCrop(224),
                  # 중앙을 224 * 224 이미지로 잘라냄 (증강 없이 고정된 영역 사용)

                  transforms.ToTensor(),
                  transforms.Normalize(mean = (0.485, 0.456, 0.406),
                                      std = (0.229, 0.224, 0.225))
                  # RGB 각 채널의 평균과 표준편차
              ]
              )

Cifar-10 데이터셋 다운로드

In [ ]:
dataset_train_full =  datasets.CIFAR10(
                      root = '/tmp/cifar.tl',
                      train = True,
                      download=True,
                      transform = transform_train
                  )

dataset_test       =  datasets.CIFAR10(
                      root = '/tmp/cifar.tl',
                      train = False,
                      download=True,
                      transform = transform_test
                  )

In [ ]:
print(f'전체 학습 데이터: {len(dataset_train_full):,}개')
print(f'전체 테스트 데이터: {len(dataset_test):,}개')
print(f'클래스 수: 10개 (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)')

소규모 서브셋 생성

- 전체 50,000개 중 각 클래스 당 300개씩, 총 3,000개만 사용하여 소규모 데이터 셋 상황 어떻게 문제 해결할 것인가?

In [ ]:
# 각 클래스별로 300개씩 선택 >> 총 3000개 서브셋 생성

selected_indices = []

# 각 클래스(0-9) 별로 카운트 저장할 딕셔너리 초기화
class_counts = {i: 0 for i in range(10)}

# class_counts
# {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0}

# 전체 학습 데이터 돌면서 각 클래스 당 300개씩 선택
for idx, (image, label) in enumerate(dataset_train_full):
  # 조건 1: 해당 클래스 현재 개수가 300개 미만이면
  if class_counts[label] < 300:
    selected_indices.append(idx)
    class_counts[label] += 1

  # 조건 2: 중지조건
  # 총 3000개 모두 선택하면 중지
  if len(selected_indices) >= 3000:
    break

# print(selected_indices)

# 선택된 인덱스(selected_indices) >> 서브셋 생성
dataset_train_small = Subset(dataset_train_full, selected_indices)

print(len(dataset_train_small))


In [ ]:
# class_counts
for class_id, count in class_counts.items():
  class_name = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                'dog','frog','horse','ship','truck'][class_id]
  print(f'{class_id} ({class_name}): {count}개')

데이터 로더 생성

In [ ]:
train_loader =  DataLoader(
                dataset_train_small,
                batch_size = 64,
                shuffle=True,
                # 한번 학습할 때 마다(매 epoch) 무작위 데이터 순서를 섞어줌 (과적합 방지)
                num_workers=2,
                pin_memory=True
            )


test_loader =   DataLoader(
                dataset_test,
                batch_size = 128,
                # 일반적으로 평가할 때 조금 더 큰 배치 사용 가능
                shuffle=False,
                # 평가시 순서 섞지 않음
                num_workers=2,
                pin_memory=True
            )

In [ ]:
# 데이터 로더 정보 출력
print(f'\n학습 데이터:')
print(f'  - 총 샘플 수: {len(dataset_train_small):,}개')
print(f'  - 배치 크기: 64')
print(f'  - 배치 수: {len(train_loader)}개')
print(f'\n테스트 데이터:')
print(f'  - 총 샘플 수: {len(dataset_test):,}개')
print(f'  - 배치 크기: 128')
print(f'  - 배치 수: {len(test_loader)}개')

모델 빌드 함수 정의

In [ ]:
# 3가지 전략 (freeze, partial, full)

def build_model(strategy = 'freeze'):
   model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
   # 사전 학습(pre-trained)된 모델의 가중치(weights) 사용

   # 원래 resnet18의 분류기 입력 특징 차원 저장
   in_features = model.fc.in_features # 512차원

   # 분류기를 CIFAR 10 에 맞게 교체 (1000 클래스 >> 10 클래스)
   model.fc = nn.Linear(in_features, 10)

   # 전략에 따라 파라미터 동결 설정
   if strategy == 'freeze':
     # 전략 1: 백본 전체를 동결 >> 분류기만 학습시키는 전략
      print('[freeze 전략]: 백본 전체를 동결 >> 분류기만 학습')

      # layer1,2,3,4 모두 동결
      for param in model.layer1.parameters():
        param.requires_grad = False # 기울기(gradient) 계산 비활성화
      for param in model.layer2.parameters():
        param.requires_grad = False # 기울기(gradient) 계산 비활성화
      for param in model.layer3.parameters():
        param.requires_grad = False # 기울기(gradient) 계산 비활성화
      for param in model.layer4.parameters():
        param.requires_grad = False # 기울기(gradient) 계산 비활성화

      # fc(분류기: classifier) 새로 생성했기 때문에 자동으로 requires_grad = True
   elif strategy == 'partial':
      # 전략 2: 마지막 블록(layer4)과 분류기만 학습
       print('[partial 전략]: 마지막 블록(layer4)과 분류기만 학습')

       # 먼저, 모든 파라미터 동결
       for param in model.parameters():
         param.requires_grad = False

       # layer4와 fc(분류기) 만 해제
       for param in model.layer4.parameters():
         param.requires_grad = True    # 기울기(gradient) 계산 활성화
       for param in model.fc.parameters():
         param.requires_grad = True

   elif strategy == 'full':
      # 전략 3: 모든 층을 학습
       print('[full 전략]: 모든 층을 학습')

       for param in model.parameters():
         param.requires_grad = True

   else:
    raise ValueError('지원되지 않는 전략입니다.')

   model = model.to(device)

   # 학습 가능한 파라미터 수 계산
   trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
   total_params = sum(p.numel() for p in model.parameters())
   print(f'학습 가능한 파라미터 : {trainable_params:,} / {total_params:,}')

   return model

학습 및 평가 함수 정의

In [ ]:
def train_and_evaluate(strategy):

  # 1.전략에 맞는 모델 생성
  model = build_model(strategy)

  # 2.최적화 함수 설정(차등 학습률 적용)
  # 헤드(분류기)와 백본의 파라미터 분리
  head_params = list(model.fc.parameters()) # 분류기 파라미터

  # 백본 파라미터 (fc 아니면서 학습가능한 파라미터)
  backbone_params = [
      param for name, param in model.named_parameters()
      if 'fc' not in name and param.requires_grad
  ]

  # 파라미터 그룹 구성(차등 학습률)
  param_groups = []

  # 백본 파라미터가 있다면
  if backbone_params:
    param_groups.append({
        'params': backbone_params,
        'lr': 1e-4 # 백본은 낮은 학습률(0.0001)
    }

    )

  # 헤드(분류기) 파라미터가 있다면
  if head_params:
    param_groups.append({
        'params': head_params,
        'lr': 1e-3 # 헤드(분류기) 높은 학습률 (0.001, 백본의 10배)
    })

  # AdamW 옵티마이저 사용(가중치 감쇠 포함)
  optimizer = optim.AdamW(param_groups, weight_decay=1e-4)


  # 손실 함수 정의
  criterion = nn.CrossEntropyLoss()

  # 3. 학습 루프
  num_epoch = 10

  for epoch in range(num_epoch):
    model.train()

    running_loss = 0.0 # 에폭별 손실 누적
    correct = 0 # 맞춘 개수
    total = 0   # 전체 샘플 수

    # 배치 별 학습
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # gradient 초기화
        optimizer.zero_grad()

        # 순전파
        outputs = model(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 역전파
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

        # 통계 업데이트
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # 에폭별 결과 출력
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    print(f'Epoch [({epoch+1}/{num_epoch}]'
          f'Loss: {epoch_loss:.4f}, '
          f'Train_Acc: {epoch_acc:.2f}%')

  model.eval() # 평가 모드로 전환

  correct = 0
  total = 0

  # 그래디언트 계산 비활성화(평가시에는 불필요)
  with torch.no_grad():
    for inputs, labels in test_loader:
          inputs = inputs.to(device)
          labels = labels.to(device)

          # 순전파
          outputs = model(inputs)

            # 예측값 계산
          predicted = outputs.argmax(dim=1)

            # 정확도 계산
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

    # 최종 테스트 정확도 계산
  test_accuracy = correct / total

  print(f'테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)')

  return test_accuracy

실험1 : Freeze 전략

In [ ]:
acc_freeze = train_and_evaluate('freeze')

실험 2: Partial 전략

In [ ]:
acc_partial = train_and_evaluate('partial')

In [ ]:
acc_full = train_and_evaluate('full')

최종 결과 비교 및 분석

In [ ]:
results = {
    'Freeze (백본 전체 동결)': acc_freeze,
    'Partial (layer4, fc 해제)': acc_partial,
    'Full(모든 층 학습)': acc_full
}

print(f'\n{"전략":<30} {"테스트 정확도":>15}')
print('='*70)

for strategy_name, accuracy in results.items():
    print(f'{strategy_name:<30} {accuracy*100:>14.2f}%')

In [ ]:
# 최고 성능 전략 찾기
best_strategy = max(results, key=results.get)
print(best_strategy)
best_accuracy = results[best_strategy]
print(best_accuracy)

In [ ]:
# Matplotlib 라이브러리 임포트
import matplotlib.pyplot as plt

# 전략 이름과 정확도
strategies = ['Freeze\n(백본 동결)', 'Partial\n(layer4 해제)', 'Full\n(전체 학습)']
accuracies = [acc_freeze * 100, acc_partial * 100, acc_full * 100]

# 막대 그래프 생성
plt.figure(figsize=(10, 6))
bars = plt.bar(strategies, accuracies, color=['#3498db', '#2ecc71', '#e74c3c'],
               alpha=0.8, edgecolor='black', linewidth=1.5)

# 각 막대 위에 정확도 값 표시
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}%',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

# 그래프 설정
plt.ylabel('Test Accuracy (%)', fontsize=12, fontweight='bold')
plt.title('Transfer Learning Strategy Comparison', fontsize=14, fontweight='bold')
plt.ylim([0, max(accuracies) * 1.15])  # Y축 범위 설정
plt.grid(True, axis='y', alpha=0.3, linestyle='--')

# 그래프 표시
plt.tight_layout()
plt.show()